In [69]:
import cv2
import os
import copy
import xml.etree.ElementTree as ET

In [ ]:
output_dir = 'E:/ar_fps/frames'

cap = cv2.VideoCapture(0)
i = 0

init_box = None
tracker = cv2.TrackerKCF_create()

x_start = 200
y_start = 200
width = 125
height = 125

class_direc = output_dir
tracked_boxes = []

while True:
    _, frame = cap.read()
    orig = frame.copy()
    
    if box:
        ret, box = tracker.update(frame)

        if ret:
            x, y, w, h = map(int, box)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            output_file = os.path.join(class_direc, 'img{}.jpg'.format(i))
            tracked_boxes.append([output_file, box])
            cv2.imwrite(output_file, orig)
        else:
            box = None
    
#         fps.update()
#         fps.stop()
        
#         cv2.putText(frame, "fps: {}".format(fps), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    else:
        cv2.rectangle(frame, (x_start, y_start), (x_start + width, y_start + height), (255, 0, 0), 2)

    k = cv2.waitKey(30) & 0xff
    
    if k == ord('s'):
        if box:
            box = None
        else:
    #         init_box = cv2.selectROI('Frame', frame, fromCenter=False, showCrosshair=True)
            box = (x_start, y_start, width, height)
            tracker = cv2.TrackerKCF_create()
            tracker.init(frame, box)
    #         fps = FPS().start()
    elif k == ord('1'): # point_foreward
        class_direc = os.path.join(output_dir, 'point_foreward')
        if not os.path.exists(class_direc):
            os.makedirs(class_direc)
        width = 75
        height = 75
    elif k == ord('2'): # point_up
        class_direc = os.path.join(output_dir, 'point_up')
        if not os.path.exists(class_direc):
            os.makedirs(class_direc)
        width = 75
        height = 125
    elif k == ord('3'): # curl
        class_direc = os.path.join(output_dir, 'curl')
        width = 75
        height = 75
        if not os.path.exists(class_direc):
            os.makedirs(class_direc)
    elif k == ord('q'):
        break
        
    cv2.imshow('img', frame)
    
    i += 1

cap.release()
cv2.destroyAllWindows()

In [66]:
copy_boxes = copy.deepcopy(tracked_boxes)

In [67]:
# create the file structure
def to_annotate(values):
    annot = ET.Element('annotation')
    
    filename = ET.SubElement(annot, 'filename')
    filename.text = values[0]
    
    size = ET.SubElement(annot, 'size')
    width = ET.SubElement(size, 'width')
    width.text = values[1]
    height = ET.SubElement(size, 'height')
    height.text = values[2]
    depth = ET.SubElement(size, 'depth')
    depth.text = '3'
    
    obj = ET.SubElement(annot, 'object')
    class_name = ET.SubElement(obj, 'class_name')
    class_name.text = values[3]
    name = ET.SubElement(obj, 'name')
    name.text = 'hand'
    box = ET.SubElement(obj, 'bndbox')
    xmin = ET.SubElement(box, 'xmin')
    xmin.text = values[4]
    ymin = ET.SubElement(box, 'ymin')
    ymin.text = values[5]
    xmax = ET.SubElement(box, 'xmax')
    xmax.text = values[6]
    ymax = ET.SubElement(box, 'ymax')
    ymax.text = values[7]
    
    return annot

In [68]:
for tracked_box in tracked_boxes:
    path, box = tracked_box
    parent_dirs = os.path.dirname(path)
    xmin, ymin, box_w, box_h = map(int, box)
    xmax = xmin + box_w
    ymax = ymin + box_h
    xmin = str(xmin)
    ymin = str(ymin)
    xmax = str(xmax)
    ymax = str(ymax)
    class_name = os.path.basename(parent_dirs)
    image_name = os.path.basename(path)
    file_name = image_name[:-3] + 'xml'
    annotation_parent_dirs = parent_dirs + '_annotations'
    if not os.path.exists(annotation_parent_dirs):
        os.makedirs(annotation_parent_dirs)
    output_path = os.path.join(annotation_parent_dirs, file_name)
    width = '640'
    height = '480'

    values = [image_name, width, height, class_name, xmin, ymin, xmax, ymax]
#     print(values)
    annot = to_annotate(values)
    data = ET.tostring(annot).decode('utf-8')
    with open(output_path, 'w') as f:
        f.write(data)

<Element 'annotation' at 0x0000016558496228>